In [1]:
from modelUnetVGG16 import *
from data import *
import os, os.path

vid="v007_5" #version id should match the file number. Last number shows the cross-validation fold number
# Ran on DellWS with GeForce RTX3060 GPU

# Count the number of train and valid files
train_dir = 'mass_seg_08/train0'+vid[-1]+'/mg'
train_count=len([name for name in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, name))])

valid_dir = 'mass_seg_08/valid0'+vid[-1]+'/mg'
valid_count=len([name for name in os.listdir(valid_dir) if os.path.isfile(os.path.join(valid_dir, name))])

### Train with data generator

In [2]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger

#Data augmentation
data_gen_args = dict(rotation_range=90,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='wrap')

# SETTINGS ***
batch_size=2
learning_rate=1e-4

train_gen = trainGenerator(batch_size,'mass_seg_08/train0'+vid[-1],'mg','mask',data_gen_args,save_to_dir = None)
valid_gen = trainGenerator(batch_size,'mass_seg_08/valid0'+vid[-1],'mg','mask',data_gen_args,save_to_dir = None)

# train_count images are used for training, valid_count images for validating
train_steps = train_count//batch_size
valid_steps = valid_count//batch_size

# SETTINGS ***
loss=dice_loss
steps_per_epoch=3*train_steps
num_epochs=100
#num_top_filter=16

model = build_vgg16_unet()

opt = tf.keras.optimizers.Adam(learning_rate)
metrics = ["acc", dice_coef, iou]
model.compile(loss=loss, optimizer=opt, metrics=metrics)

callbacks = [
             ModelCheckpoint('files_mass_seg_xval/unet_mass_seg_'+vid+'.hdf5', verbose=1, save_best_model=True),
             ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.1, verbose=1, min_lr=1e-8),
             CSVLogger("files_mass_seg_xval/data_"+vid+".csv"),
             EarlyStopping(monitor="val_loss", patience=5, verbose=1)
            ]

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps, 
                    epochs=num_epochs, callbacks=callbacks)


Found 283 images belonging to 1 classes.


C:\Users\Administrator\AppData\Local\Temp\ipykernel_19064\910975809.py:43: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=steps_per_epoch, validation_steps=valid_steps,


Found 283 images belonging to 1 classes.
Epoch 1/100
423/423 [==============================] - ETA: 0s - loss: 0.8592 - acc: 0.8928 - dice_coef: 0.1408 - iou: 0.0823Found 69 images belonging to 1 classes.
Found 69 images belonging to 1 classes.

Epoch 1: saving model to files_mass_seg_xval\unet_mass_seg_v007_5.hdf5
423/423 [==============================] - 468s 906ms/step - loss: 0.8592 - acc: 0.8928 - dice_coef: 0.1408 - iou: 0.0823 - val_loss: 0.8702 - val_acc: 0.8587 - val_dice_coef: 0.1298 - val_iou: 0.0737 - lr: 1.0000e-04
Epoch 2/100
423/423 [==============================] - ETA: 0s - loss: 0.7828 - acc: 0.9588 - dice_coef: 0.2171 - iou: 0.1319
Epoch 2: saving model to files_mass_seg_xval\unet_mass_seg_v007_5.hdf5
423/423 [==============================] - 346s 818ms/step - loss: 0.7828 - acc: 0.9588 - dice_coef: 0.2171 - iou: 0.1319 - val_loss: 0.8154 - val_acc: 0.9017 - val_dice_coef: 0.1846 - val_iou: 0.1096 - lr: 1.0000e-04
Epoch 3/100
423/423 [============================

### validate your model and save predicted results

In [3]:
validGene = testGenerator("mass_seg_08/valid0"+vid[-1]+"/pred")
model.load_weights("files_mass_seg_xval/unet_mass_seg_"+vid+".hdf5")
results = model.predict_generator(validGene,valid_count,verbose=1)
saveResult("mass_seg_08/valid0"+vid[-1]+"/pred",results,vid[:4])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_19064\2576895886.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  results = model.predict_generator(validGene,valid_count,verbose=1)


69/69 [==============================] - 15s 207ms/step


D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\1_predict_v007.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\3_predict_v007.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\4_predict_v007.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\5_predict_v007.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"%(i,vid)),img)
D:\research\breastCancer\mass_seg\codes\data.py:110: UserWarning: mass_seg_08/valid05/pred\6_predict_v007.png is a low contrast image
  io.imsave(os.path.join(save_path,"%d_predict_%s.png"